In [1]:
from bs4 import BeautifulSoup, SoupStrainer
import urllib
from urlparse import urljoin
import urlparse
from gensim import corpora
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

mydict = {}
map_url_text = {}
tokens = []
inverted = {}

# store homepage url:text dict 'mydict'
url = 'https://www.macmillan.org.uk'
response = urllib.urlopen(url).read()
soup1 = BeautifulSoup(response, "lxml")
text = soup1.get_text()
mydict[url] = text

# extract links from homepage text and store them in list with unique elements 'links'
soup2 = BeautifulSoup(response, parse_only=SoupStrainer('a', href=True))
links = soup2.find_all('a')
links = [s.get('href') for s in links]
links = [unicode(s) for s in links]
links = [s for s in links if (s.startswith("https://") or s.startswith("http://"))]
# links = set(links)

# fetch text for each url in 'links' and store them in dict 'map_url_text'
for l in links:
    r = urllib.urlopen(l).read()
    soup = BeautifulSoup(r, "lxml")
    for s in soup(["script", "style"]):
        s.extract()
    t = soup.get_text()
    lines = (line.strip() for line in t.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    t = ' '.join(chunk for chunk in chunks if chunk)
    map_url_text[l] = t

# tokenize text in 'map_url_text' and add unique tokens to list 'tokens'
for key, value in map_url_text.iteritems():
    value = set(value.split())
    for word in value:
        tokens.append(word)

# build inverted index
# for k in tokens:
#     for key, value in map_url_text.iteritems():
#         if k in value:
#             if any(k is term for term, urls in inverted.iteritems()):
#                 if key not in inverted[k]:
#                     inverted[k].append(key) 
#             else:
#                 inverted[k] = [key]

# print inverted

/Applications/anaconda/lib/python2.7/site-packages/bs4/__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


In [2]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [7]:
from gensim import corpora, models, similarities
from pprint import pprint 
from nltk.corpus import stopwords


# 'documents' are list of texts
documents = (value for key, value in map_url_text.iteritems())

# remove common words and tokenize
stoplist = set(stopwords.words('english')) and ('@ + 0 1 2 3 4 5 6 7 8 9 / . , = - '.split()) 
texts = [[word for word in document.lower().split() if word not in stoplist] 
         for document in documents]

# assign unique ids to unique tokens from texts
dictionary = corpora.Dictionary(texts)
# dictionary.save('/example.dict')  # store the dictionary, for future reference

# build inverted index from dictionary
# for tok in dictionary:
#     for key, value in map_url_text.iteritems():
#         if dictionary[tok] in value:
#             if any(dictionary[tok] is term for term, urls in inverted.iteritems()):
#                 if key not in inverted[(dictionary[tok])]:
#                     inverted[(dictionary[tok])].append(key) 
#             else:
#                 inverted[(dictionary[tok])] = [key]

In [ ]:
# build positional index
for tok in dictionary:
    for key, value in map_url_text.iteritems():
        for word in value:
            if dictionary[tok] is word:
                d = {key:[value.index(word)]}
                if any(dictionary[tok] is term for term, urls in inverted.iteritems()):
                    if key not in inverted[(dictionary[tok])]:
                        inverted[(dictionary[tok])].append(d) 
                else:
                    inverted[(dictionary[tok])] = [d]
                    
print inverted

In [7]:
positional = {}

myDict = {}
myDict['1'] = 'hello'
myDict['2'] = 'bye'
myDict['3'] = 'school'
myDict['4'] = 'uni'
myDict['5'] = 'London'

mapUrlText = {}
mapUrlText['doc1'] = 'hello this is london'
mapUrlText['doc2'] = 'the school is nice'
mapUrlText['doc3'] = 'uni and school'
mapUrlText['doc4'] = 'bye London'

# for tok, item in myDict.iteritems():
#     for key, value in mapUrlText.iteritems():
#         for word in value:
#             if item == word:
#                 d = {key:[value.index(word)]}
#                 if any(item is term for term, urls in positional.iteritems()):
#                     if key not in positional[item]:
#                         positional[item].append(d)
#                 else:
#                     positional[item] = [d]

#  for tok, item in myDict.iteritems():
#     for key, value in mapUrlText.iteritems():
#         for word in value.split():
#             if item is word:
#                 print key, value.index(word)

for tok, item in myDict.iteritems():
    for key, value in mapUrlText.iteritems():
#         print len(value.split())
        for word in value.split():
            if item == word:
                print word, key, value.index(word)

hello doc1 0
school doc2 4
school doc3 8
bye doc4 0
London doc4 4
uni doc3 0


In [30]:
import math
from decimal import *
import operator

# map tokens<->number of docs in which they appear

map_tokens_in_docs = {}
dictionary_bis = {}
special_chars = '@  www « ( ) + 0 1 2 3 4 5 6 7 8 9 / . , = - £ $ ^ %'

for tok in dictionary:
    if ((dictionary[tok].startswith("https://") or dictionary[tok].startswith("http://") 
         or dictionary[tok].startswith('@') or dictionary[tok].startswith('www')) is False):
        dictionary_bis[tok] = dictionary[tok]

for tok in dictionary_bis:
    docs = []
    for k_url, value in map_url_text.iteritems():
        if dictionary_bis[tok] in value:
            docs.append(k_url)
        map_tokens_in_docs[dictionary_bis[tok]] = len(docs)

# for item in map_tokens_in_docs:
#     print map_tokens_in_docs[item]
    
# tfidf = term_freq * (log(total_docs/docs_with_term))
# map token<->[url:tfidf_score]


map_tokens_listUrlTfidf = {}

for tok, list_docs in map_tokens_in_docs.iteritems():
    list_urls_score = {}
    for key, value in map_url_text.iteritems():
        term_freq = value.count(tok)
        getcontext().prec = 9
        idf = Decimal(len(map_url_text))/Decimal(1+list_docs)
        log_idf = math.log(idf, 2)
        score_tfidf = Decimal(term_freq*log_idf)
        list_urls_score[key] = score_tfidf
    map_tokens_listUrlTfidf[tok] = list_urls_score

# for t, lst in map_tokens_listUrlTfidf.iteritems():
#     sorted_lst = sorted(lst.items(), key=operator.itemgetter(1), reverse=True)
#     print t, sorted_lst

for t, lst in map_tokens_listUrlTfidf.iteritems():
    for url in lst:
#         if t in url:
        print t, url

In [ ]:
from nltk.stem.porter import *



def search(user_query):
    mylist = []
    # tokenize and stem user input
    string = user_query.split()
    stemmer = PorterStemmer()
    for w in string:
        stemmer.stem(w)
    
    # check if present in index
    for i in string:
        for t, lst in map_tokens_listUrlTfidf.iteritems():
            if i in t:
                for key in sorted(lst, reverse=True)[:10]:
                    if (lst[key] != 0) or (i in key):
                        mylist.append(key)
                        
    mylist = set(mylist)
    for i in mylist:
        print i    

In [50]:
if __name__ == '__main__':
    search('community support')

http://www.youtube.com/user/macmillancancer
http://www.macmillan.org.uk/information-and-support/understanding-cancer/about-our-website.html
https://my.macmillan.org.uk/Account/LogOn?returnurl=https://www.macmillan.org.uk/
https://my.macmillan.org.uk/Profile/SavedPages
https://www.facebook.com/macmillancancer
http://www.pinterest.com/macmillancancer
http://www.macmillan.org.uk/get-involved/fundraising-events/index.html
http://www.macmillan.org.uk/get-involved/all-ways-to-help.html
https://twitter.com/macmillancancer
https://my.macmillan.org.uk/Account/Register?src=sso?returnurl=https://www.macmillan.org.uk/


In [51]:
if __name__ == '__main__':
    search('funraising events')

http://www.macmillan.org.uk/get-involved/fundraising-events/index.html
https://my.macmillan.org.uk/Account/LogOn?returnurl=https://www.macmillan.org.uk/
https://my.macmillan.org.uk/Account/Register?src=sso?returnurl=https://www.macmillan.org.uk/
http://www.macmillan.org.uk/get-involved/all-ways-to-help.html
https://www.facebook.com/macmillancancer


In [52]:
if __name__ == '__main__':
    search('medical facility')

http://www.macmillan.org.uk/get-involved/all-ways-to-help.html
